In [2]:
# General Libraries
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
import time
import re
import requests
import pickle
import seaborn as sns
import os
import glob
import sys
#from natsort import natsorted
sns.set()

import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

import datetime
from datetime import timedelta, date 
start = time.time()
%matplotlib inline

# Nueral Networks are stocastic; setting random seeds ensures we get the same output each time
SEED = 42
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # TF 2.1
np.random.seed(SEED)
#tf.random.set_seed(SEED)
import random as python_random
python_random.seed(2250)

print('Libraries imported')

Libraries imported


In [3]:
train1 = pd.read_excel("/kaggle/input/idareai-bipro/CO2_Injection_rate train.xlsx")
inj_diff_train = train1.iloc[:,1]
train1 = train1.drop(train1.columns[1:2], axis=1)
train1["inj_diff"] = inj_diff_train
print(inj_diff_train.shape)

#train = train.drop(train.columns[0:3], axis=1)
#print(train.info())
#display(train1.head())

test_1 = pd.read_excel("/kaggle/input/idareai-bipro/CO2_Injection_rate test exam.xlsx")
#test = test.drop(test.columns[1], axis=1)
test_1["inj_diff"] = 0
#print(train.info())
#display(test_1.head())

(1715,)


# Loading complete full leangth dataset.

In [4]:
print(train1.columns)
print(test_1.columns)
print(train1.shape)
print(test_1.shape)

Index(['Rand', 'Date Time', 'Avg_PLT_CO2VentRate_TPH',
       'Avg_CCS1_WHCO2InjPs_psi', 'Avg_CCS1_WHCO2InjTp_F', 'Avg_CCS1_ANPs_psi',
       'Avg_CCS1_DH6325Ps_psi', 'Avg_CCS1_DH6325Tp_F', 'Avg_VW1_WBTbgPs_psi',
       'Avg_VW1_WBTbgTp_F', 'Avg_VW1_ANPs_psi', 'Avg_VW1_Z11D4917Ps_psi',
       'Avg_VW1_Z11D4917Tp_F', 'Avg_VW1_Z10D5001Ps_psi',
       'Avg_VW1_Z10D5001Tp_F', 'Avg_VW1_Z09D5653Ps_psi',
       'Avg_VW1_Z09D5653Tp_F', 'Avg_VW1_Z08D5840Ps_psi',
       'Avg_VW1_Z08D5840Tp_F', 'Avg_VW1_Z07D6416Ps_psi',
       'Avg_VW1_Z07D6416Tp_F', 'Avg_VW1_Z06D6632Ps_psi',
       'Avg_VW1_Z06D6632Tp_F', 'Avg_VW1_Z04D6837Ps_psi',
       'Avg_VW1_Z04D6837Tp_F', 'Avg_VW1_Z03D6945Ps_psi',
       'Avg_VW1_Z03D6945Tp_F', 'Avg_VW1_Z02D6982Ps_psi',
       'Avg_VW1_Z02D6982Tp_F', 'Avg_VW1_Z01D7061Ps_psi',
       'Avg_VW1_Z01D7061Tp_F', 'Avg_VW1_Z0910D5482Ps_psi',
       'Avg_VW1_Z0910D5482Tp_F', 'inj_diff'],
      dtype='object')
Index(['Rand', 'Date Time', 'Avg_PLT_CO2VentRate_TPH',
       'Avg_CCS1_W

In [5]:
dfc = pd.concat([train1, test_1], axis=0)
dfc.shape

(1864, 34)

In [6]:
from itertools import combinations

def get_temperature_diff_depth(df):
    # Define depth levels and their corresponding column names
    depth_cols = {
        7061: 'Avg_VW1_Z01D7061Tp_F',
        6982: 'Avg_VW1_Z02D6982Tp_F',
        6837: 'Avg_VW1_Z04D6837Tp_F',
        6632: 'Avg_VW1_Z06D6632Tp_F',
        6416: 'Avg_VW1_Z07D6416Tp_F',
        5840: 'Avg_VW1_Z08D5840Tp_F',
        5482: 'Avg_VW1_Z0910D5482Tp_F',
        5653: 'Avg_VW1_Z09D5653Tp_F',
        5001: 'Avg_VW1_Z10D5001Tp_F',
        4917: 'Avg_VW1_Z11D4917Tp_F'
    }
    
    # Generate temperature differences dynamically
    for (depth1, depth2) in combinations(depth_cols.keys(), 2):
        col_name = f'Temperature diff {depth1}-{depth2} ft'
        df[col_name] = df[depth_cols[depth1]] - df[depth_cols[depth2]]

    return df

def get_pressure_diff_depth(df):
    depth_columns = {
        7061: 'Avg_VW1_Z01D7061Ps_psi',
        6982: 'Avg_VW1_Z02D6982Ps_psi',
        6837: 'Avg_VW1_Z04D6837Ps_psi',
        6632: 'Avg_VW1_Z06D6632Ps_psi',
        6416: 'Avg_VW1_Z07D6416Ps_psi',
        5840: 'Avg_VW1_Z08D5840Ps_psi',
        5482: 'Avg_VW1_Z0910D5482Ps_psi',
        5653: 'Avg_VW1_Z09D5653Ps_psi',
        5001: 'Avg_VW1_Z10D5001Ps_psi',
        4917: 'Avg_VW1_Z11D4917Ps_psi',
    }

    # Compute pressure differences dynamically
    for (depth1, col1), (depth2, col2) in combinations(depth_columns.items(), 2):
        df[f'Pressure diff {depth1}-{depth2} ft'] = df[col1] - df[col2]
    
    return df
    
def create_lag_features(df, features, n_lags=3):
    """
    Create lag features, percentage change, and difference features for the given dataframe and feature columns.
    """

    lagged_df = pd.DataFrame()

    # Ensure we only apply to numeric columns
    numeric_cols = df[features].select_dtypes(include=[np.number]).columns

    for feature in numeric_cols:
        for lag in range(1, n_lags + 1):
            # Create lagged features
            lagged_feature = f"{feature}_lag{lag}"
            lagged_df[lagged_feature] = df[feature].shift(lag)

            # Create percentage change features
            pct_change_feature = f"{feature}_pct_change{lag}"
            lagged_df[pct_change_feature] = df[feature].pct_change(lag)

            # Create difference features
            diff_feature = f"{feature}_diff{lag}"
            lagged_df[diff_feature] = df[feature].diff(lag)

    return lagged_df



def create_abs_features(df, features):
    """
    Create absolute value and log absolute value features for the given dataframe and feature columns.
    """
    # Ensure we only apply to numeric columns (exclude datetime columns)
    numeric_cols = df[features].select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        df['Abs ' + col] = np.abs(df[col])
        df['Log abs ' + col] = np.log(np.abs(df[col]) + 1e-7)  # Adding small epsilon to avoid log(0)

    return df


def create_trend_features(df, features):
    """
    Create trend-based features such as Simple Moving Average (SMA) and Exponential Moving Average (EMA) for the given dataframe and feature columns.
    """
    # Filter only numeric columns for rolling operations
    numeric_cols = df[features].select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        # Add simple moving average (SMA) with a window of 5
        df[col + ' sma 5'] = df[col].rolling(window=5).mean()

        # Add exponential moving average (EMA) with a span of 5
        df[col + ' ema 5'] = df[col].ewm(span=5, adjust=False).mean()

    return df


In [7]:
dfc = get_temperature_diff_depth(dfc)
dfc = get_pressure_diff_depth(dfc)

dfc['Diff pressure downhole-wellhead'] = dfc['Avg_CCS1_DH6325Ps_psi'] - dfc['Avg_CCS1_ANPs_psi']

cols = dfc.columns
lagged_df = create_lag_features(dfc, features=cols, n_lags=5)

# Create additional features using the functions
dfc = create_abs_features(dfc, dfc.columns)
dfc = create_trend_features(dfc, dfc.columns)


In [8]:
# Remove the columns 'Rand', 'Date Time', and 'inj_diff'
dfc2 = dfc.drop(columns=['Rand', 'Date Time', 'inj_diff'])

# Verify the changes by checking the first few rows
display(dfc2)

,Avg_PLT_CO2VentRate_TPH,Avg_CCS1_WHCO2InjPs_psi,Avg_CCS1_WHCO2InjTp_F,Avg_CCS1_ANPs_psi,Avg_CCS1_DH6325Ps_psi,Avg_CCS1_DH6325Tp_F,Avg_VW1_WBTbgPs_psi,Avg_VW1_WBTbgTp_F,Avg_VW1_ANPs_psi,Avg_VW1_Z11D4917Ps_psi,...,Log abs Pressure diff 5653-4917 ft sma 5,Log abs Pressure diff 5653-4917 ft ema 5,Abs Pressure diff 5001-4917 ft sma 5,Abs Pressure diff 5001-4917 ft ema 5,Log abs Pressure diff 5001-4917 ft sma 5,Log abs Pressure diff 5001-4917 ft ema 5,Abs Diff pressure downhole-wellhead sma 5,Abs Diff pressure downhole-wellhead ema 5,Log abs Diff pressure downhole-wellhead sma 5,Log abs Diff pressure downhole-wellhead ema 5
0,0.0,1105.445866,71.965004,591.478453,3320.663664,107.016855,2232.331462,104.056371,5.289917,2073.506816,...,NaN,6.132539,NaN,33.876664,NaN,3.522726,NaN,2729.185211,NaN,7.911758
1,0.0,1096.617687,72.497477,570.684185,3318.985713,106.693605,2232.333350,104.053463,5.231462,2073.517420,...,NaN,6.132513,NaN,33.873635,NaN,3.522637,NaN,2735.557317,NaN,7.914085
2,0.0,1096.485427,73.214516,565.322360,3322.928800,106.402567,2232.451115,104.090451,5.173645,2073.483464,...,NaN,6.132485,NaN,33.877325,NaN,3.522746,NaN,2742.907024,NaN,7.916763
3,0.0,1105.192650,76.607459,620.087081,3315.569511,106.411401,2232.515324,104.098971,5.162659,2073.451095,...,NaN,6.132478,NaN,33.877008,NaN,3.522737,NaN,2727.098826,NaN,7.910953
4,0.0,1114.396364,76.466836,686.325817,3315.742818,106.941482,2232.518034,104.089386,5.120544,2073.428835,...,6.132481,6.132489,33.880715,33.884090,3.522846,3.522946,2711.998522,2694.538218,7.905293,7.898808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.0,788.978516,39.225381,545.660900,2999.303609,115.228923,2231.929484,104.094711,2.752075,2073.523604,...,6.137724,6.136096,33.526780,33.591713,3.512329,3.514268,2574.286173,2545.512571,7.852916,7.841680
145,0.0,1073.501234,86.192300,667.184589,3152.468219,123.804093,2230.813951,104.186301,1.665344,2073.610128,...,6.136245,6.136277,33.583709,33.580645,3.514033,3.513942,2536.029976,2525.436257,7.838087,7.833834
146,0.0,1233.839675,90.415624,573.751962,3246.029901,124.836247,2210.984052,103.393133,1.261628,2056.167855,...,6.134388,6.134826,33.521877,33.405556,3.512171,3.508690,2547.586039,2574.383484,7.842459,7.852785
147,0.0,1148.058396,83.329263,546.711668,3351.503379,119.379877,2228.566459,104.299927,0.602294,2073.673769,...,6.135944,6.138072,33.469421,33.366729,3.510601,3.507536,2597.827222,2651.186226,7.861225,7.881551


In [9]:
def remove_redundant_features(df, threshold):
    correlation_matrix = df.corr().abs()

    to_remove = set()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if correlation_matrix.iloc[i, j] > threshold: 
                colname = correlation_matrix.columns[i] 
                to_remove.add(colname) 

    df_reduced = df.drop(columns=to_remove)
    return df_reduced, to_remove

dfc2_reduced, removed_features = remove_redundant_features(dfc2, threshold=0.8)

#print("Removed Features:", removed_features)

display(dfc2_reduced.shape)
display(dfc2_reduced)
#display(train.head())

(1864, 32)

,Avg_PLT_CO2VentRate_TPH,Avg_CCS1_WHCO2InjPs_psi,Avg_CCS1_ANPs_psi,Avg_CCS1_DH6325Ps_psi,Avg_CCS1_DH6325Tp_F,Avg_VW1_WBTbgPs_psi,Avg_VW1_WBTbgTp_F,Avg_VW1_ANPs_psi,Temperature diff 6982-6632 ft,Temperature diff 6837-5482 ft,...,Avg_CCS1_ANPs_psi sma 5,Avg_VW1_WBTbgPs_psi sma 5,Avg_VW1_WBTbgTp_F sma 5,Avg_VW1_Z11D4917Ps_psi sma 5,inj_diff sma 5,inj_diff ema 5,Temperature diff 5482-4917 ft sma 5,Temperature diff 5653-4917 ft sma 5,Abs inj_diff sma 5,Abs Diff pressure downhole-wellhead sma 5
0,0.0,1105.445866,591.478453,3320.663664,107.016855,2232.331462,104.056371,5.289917,1.260313,7.484037,...,NaN,NaN,NaN,NaN,NaN,1.100694e-01,NaN,NaN,NaN,NaN
1,0.0,1096.617687,570.684185,3318.985713,106.693605,2232.333350,104.053463,5.231462,1.271176,7.488086,...,NaN,NaN,NaN,NaN,NaN,-4.533637e-02,NaN,NaN,NaN,NaN
2,0.0,1096.485427,565.322360,3322.928800,106.402567,2232.451115,104.090451,5.173645,1.273078,7.481141,...,NaN,NaN,NaN,NaN,NaN,-8.153137e-02,NaN,NaN,NaN,NaN
3,0.0,1105.192650,620.087081,3315.569511,106.411401,2232.515324,104.098971,5.162659,1.266433,7.479446,...,NaN,NaN,NaN,NaN,NaN,-1.994931e-01,NaN,NaN,NaN,NaN
4,0.0,1114.396364,686.325817,3315.742818,106.941482,2232.518034,104.089386,5.120544,1.258258,7.455356,...,606.779579,2232.429857,104.077729,2073.477526,-0.039167,8.019897e-02,6.985804,8.012388,0.339028,2711.998522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.0,788.978516,545.660900,2999.303609,115.228923,2231.929484,104.094711,2.752075,NaN,NaN,...,593.429734,2230.809854,104.210440,2073.613184,0.000000,9.819077e-27,7.088492,7.997302,0.000000,2574.286173
145,0.0,1073.501234,667.184589,3152.468219,123.804093,2230.813951,104.186301,1.665344,1.224226,7.378085,...,617.855036,2231.176225,104.189845,2073.585923,0.000000,6.546052e-27,7.075407,7.993438,0.000000,2536.029976
146,0.0,1233.839675,573.751962,3246.029901,124.836247,2210.984052,103.393133,1.261628,1.220156,7.280089,...,605.581403,2227.426100,104.005090,2070.077507,0.000000,4.364034e-27,7.058278,7.966176,0.000000,2547.586039
147,0.0,1148.058396,546.711668,3351.503379,119.379877,2228.566459,104.299927,0.602294,1.240546,7.298390,...,577.326613,2226.945674,104.016952,2070.090192,0.000000,2.909356e-27,7.059027,7.969161,0.000000,2597.827222


In [10]:
# Separate first 1715 rows, as these are the train files
df_first_part = dfc2_reduced.iloc[:1715]

# Separate the next 149 rows, as these are teh test files
df_second_part = dfc2_reduced.iloc[1715:1864]
# Save the separated DataFrames as CSV files
df_first_part.to_csv("train_selected_features.csv", index=False)
df_second_part.to_csv("test_selected_features.csv", index=False)

# Verify by checking the first few rows of each DataFrame
print("First part DataFrame:")
print(df_first_part.shape)

print("\nSecond part DataFrame:")
print(df_second_part.shape)


First part DataFrame:
(1715, 32)

Second part DataFrame:
(149, 32)


In [11]:
X = df_first_part
X.fillna(X.mean(), inplace=True) 
#print(X.head())
y = inj_diff_train

In [12]:
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib

# Split data into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features (optional, but helps with some models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
print(X_train_scaled.shape)
X_test_scaled = scaler.transform(X_test)
# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42),
    "LightGBM": LGBMRegressor(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42),
    "Support Vector Regression": SVR(kernel="rbf"),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "CatBoost": CatBoostRegressor(iterations=500, learning_rate=0.05, depth=5, random_state=42, verbose=0),
    "Elastic Net": ElasticNet(alpha=1.0, l1_ratio=0.5),
    "KNN": KNeighborsRegressor(n_neighbors=5),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "MLP Neural Network": MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42),
    "Bayesian Ridge": BayesianRidge(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Extra Trees": ExtraTreesRegressor(n_estimators=100, random_state=42)
}



(1372, 32)


In [13]:

# Train and save models
results = []
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    
    # Save the trained model
    joblib.dump(model, f"{name.replace(' ', '_').lower()}.pkl")
    
    # Predict on test data
    y_pred = model.predict(X_test_scaled)
    
    # Evaluate
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    results.append((name, r2, mae, rmse))
    print(f"🔹 Model: {name}")
    print(f"   R² Score: {r2:.4f}")
    print(f"   Mean Absolute Error: {mae:.4f}")
    print(f"   Root Mean Squared Error: {rmse:.4f}\n")

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=["Model", "R² Score", "MAE", "RMSE"])
print(results_df.sort_values(by="R² Score", ascending=False))

🔹 Model: Linear Regression
   R² Score: 0.6693
   Mean Absolute Error: 1.1850
   Root Mean Squared Error: 2.5709

🔹 Model: Random Forest
   R² Score: 0.9059
   Mean Absolute Error: 0.4562
   Root Mean Squared Error: 1.3715

🔹 Model: XGBoost
   R² Score: 0.9588
   Mean Absolute Error: 0.3521
   Root Mean Squared Error: 0.9072

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000378 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7672
[LightGBM] [Info] Number of data points in the train set: 1372, number of used features: 32
[LightGBM] [Info] Start training from score 0.019272
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

In [14]:
# Load saved models
dt_model = joblib.load("/kaggle/working/decision_tree.pkl")
xgb_model = joblib.load("/kaggle/working/xgboost.pkl")
gb_model = joblib.load("/kaggle/working/gradient_boosting.pkl")

#predict again
y_pred_dt = dt_model.predict(X_test_scaled)
y_pred_xgb = xgb_model.predict(X_test_scaled)
y_pred_gb = gb_model.predict(X_test_scaled)

# Define weights based on previous R² scores
w_dt, w_xgb, w_gb = 0.34, 0.33, 0.33  

# Apply Ensemble Method (Weighted Prediction)
y_pred_ensemble = (w_dt * y_pred_dt) + (w_xgb * y_pred_xgb) + (w_gb * y_pred_gb)

# Evaluate ensemble model
r2_ensemble = r2_score(y_test, y_pred_ensemble)
mae_ensemble = mean_absolute_error(y_test, y_pred_ensemble)
rmse_ensemble = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))

# Print ensemble results
print(f"🔹 Weighted Ensemble Model")
print(f"   R² Score: {r2_ensemble:.4f}")
print(f"   Mean Absolute Error: {mae_ensemble:.4f}")
print(f"   Root Mean Squared Error: {rmse_ensemble:.4f}")


🔹 Weighted Ensemble Model
   R² Score: 0.9708
   Mean Absolute Error: 0.3326
   Root Mean Squared Error: 0.7644


In [15]:
# Preprocess the second dataset
X_second_part = df_second_part.copy()
X_second_part.fillna(X.mean(), inplace=True)  # Fill missing values with mean of the first dataset

# Scale using the previously fitted scaler
X_second_part_scaled = scaler.transform(X_second_part)

# Make predictions using trained models
y_pred_dt_second = dt_model.predict(X_second_part_scaled)
y_pred_xgb_second = xgb_model.predict(X_second_part_scaled)
y_pred_gb_second = gb_model.predict(X_second_part_scaled)

# Compute ensemble prediction using the same weights
y_pred_ensemble_second = (w_dt * y_pred_dt_second) + (w_xgb * y_pred_xgb_second) + (w_gb * y_pred_gb_second)

# Print predictions
print("🔹 Ensemble Predictions for the Second Dataset:")
print(y_pred_ensemble_second.shape)

🔹 Ensemble Predictions for the Second Dataset:
(149,)


In [16]:
y_pred_ensemble_second

array([ 0.04357618,  0.02819521,  0.09050736,  0.01696236,  0.17769035,
        0.13776548,  0.05792838,  0.03369466,  0.0450121 ,  0.11355072,
        0.10296725,  0.21583537,  0.0796549 ,  0.057229  ,  0.13242442,
        0.11422423,  0.09715326,  0.13902168,  0.060035  ,  0.08434491,
        0.11418469,  0.19574223,  0.07246631,  0.14352077,  0.08245305,
        0.1059036 ,  0.13952238,  0.0137447 ,  0.11022285,  0.04647233,
       -0.02453808,  0.16180684, -0.05766126,  0.07189168,  0.1430053 ,
       -0.00061082,  0.1426161 , -0.06455665,  0.13917619,  0.10850284,
        0.10353865,  0.1719077 ,  0.21379989,  0.09310958,  0.01904368,
        0.04874089,  0.08801302,  0.15905027,  0.16638162,  0.1238526 ,
        0.10555671,  0.0392322 ,  0.05485469,  0.13404904,  0.15060176,
        0.15967993,  0.06398949,  0.02386027,  0.06267429,  0.05744913,
        0.09036601,  0.11042535,  0.11973463,  0.14420968,  0.14204423,
        0.02274808,  0.03812048,  0.09292344,  0.04742936,  0.03

In [17]:
# Preprocess df_second_part
X_second_part = df_second_part.copy()
X_second_part.fillna(X_second_part.mean(), inplace=True)
X_second_part_scaled = scaler.transform(X_second_part)

# Predict using loaded models
y_pred_dt = dt_model.predict(X_second_part_scaled)
y_pred_xgb = xgb_model.predict(X_second_part_scaled)
y_pred_gb = gb_model.predict(X_second_part_scaled)

# Define weights based on previous R² scores
w_dt, w_xgb, w_gb = 0.34, 0.33, 0.33  

# Apply Ensemble Method (Weighted Prediction)
y_pred_ensemble = (w_dt * y_pred_dt) + (w_xgb * y_pred_xgb) + (w_gb * y_pred_gb)

# Save predictions
df_second_part["Ensemble_Prediction"] = y_pred_ensemble
df_second_part.to_csv("second_part_predictions.csv", index=False)

print("✅ Predictions saved in 'second_part_predictions.csv'")

✅ Predictions saved in 'second_part_predictions.csv'


In [18]:
test_1_filtered = test_1[['Rand', 'Date Time']]
test_1_filtered['inj_diff'] = y_pred_ensemble  # Assuming 'y_pred_ensemble' is your final ensemble prediction

# Step 4: Save the modified DataFrame to a new CSV file
test_1_filtered.to_csv('Ensemble_CO2_inject_rate_test_exam.csv', index=False)

print("File saved as 'Ensemble_CO2_inject_rate_test_exam.csv'.")

File saved as 'Ensemble_CO2_inject_rate_test_exam.csv'.


In [19]:
display(test_1_filtered)

,Rand,Date Time,inj_diff
0,196,2024-12-03 18:00:00,0.043576
1,199,2025-01-25 21:00:00,0.028195
2,434,2024-12-16 21:00:00,0.090507
3,443,2024-12-08 19:00:00,0.016962
4,644,2025-01-22 09:00:00,0.177690
...,...,...,...
144,7114,2024-12-02 15:00:00,0.088045
145,7114,2024-12-21 13:00:00,0.060955
146,7155,2025-01-09 09:00:00,0.071790
147,7230,2025-02-01 04:00:00,0.062575


In [20]:
# Load the saved models
dt_model = joblib.load("/kaggle/working/decision_tree.pkl")
xgb_model = joblib.load("/kaggle/working/xgboost.pkl")
gb_model = joblib.load("/kaggle/working/gradient_boosting.pkl")
catboost_model = joblib.load("/kaggle/working/catboost.pkl")
rf_model = joblib.load("/kaggle/working/random_forest.pkl")
extra_trees_model = joblib.load("/kaggle/working/extra_trees.pkl")
lgbm_model = joblib.load("/kaggle/working/lightgbm.pkl")
knn_model = joblib.load("/kaggle/working/knn.pkl")
lasso_model = joblib.load("/kaggle/working/lasso_regression.pkl")
bayesian_ridge_model = joblib.load("/kaggle/working/bayesian_ridge.pkl")
ridge_model = joblib.load("/kaggle/working/ridge_regression.pkl")
mlp_model = joblib.load("/kaggle/working/mlp_neural_network.pkl")
elastic_net_model = joblib.load("/kaggle/working/elastic_net.pkl")
svr_model = joblib.load("/kaggle/working/support_vector_regression.pkl")


test_1_filtered2 = test_1[['Rand', 'Date Time']]

# Define the models and their predictions
models = {
    "Decision Tree": dt_model,
    "XGBoost": xgb_model,
    "Gradient Boosting": gb_model,
    "CatBoost": catboost_model,
    "Random Forest": rf_model,
    "Extra Trees": extra_trees_model,
    "LightGBM": lgbm_model,
    "KNN": knn_model,
    "Lasso Regression": lasso_model,
    "Bayesian Ridge": bayesian_ridge_model,
    "Ridge Regression": ridge_model,
    "MLP Neural Network": mlp_model,
    "Elastic Net": elastic_net_model,
    "Support Vector Regression": svr_model
}
#print(test_1_filtered)
# Iterate through each model and save its predictions to a separate file
for model_name, model in models.items():
    # Predict on the test data (scaled or unscaled depending on your model setup)
    predictions = model.predict(X_second_part_scaled)  # assuming you want to use X_test_scaled for the predictions
    
    # Add the prediction results for the current model
    test_1_filtered2['inj_diff'] = predictions
    print(1)
    # Save the result as a CSV file
    filename = f"{model_name.replace(' ', '_')}_CO2_inject_rate_test_exam.csv"
    test_1_filtered2.to_csv(filename, index=False)
    
    print(f"File saved as '{filename}'.")


1
File saved as 'Decision_Tree_CO2_inject_rate_test_exam.csv'.
1
File saved as 'XGBoost_CO2_inject_rate_test_exam.csv'.
1
File saved as 'Gradient_Boosting_CO2_inject_rate_test_exam.csv'.
1
File saved as 'CatBoost_CO2_inject_rate_test_exam.csv'.
1
File saved as 'Random_Forest_CO2_inject_rate_test_exam.csv'.
1
File saved as 'Extra_Trees_CO2_inject_rate_test_exam.csv'.
1
File saved as 'LightGBM_CO2_inject_rate_test_exam.csv'.
1
File saved as 'KNN_CO2_inject_rate_test_exam.csv'.
1
File saved as 'Lasso_Regression_CO2_inject_rate_test_exam.csv'.
1
File saved as 'Bayesian_Ridge_CO2_inject_rate_test_exam.csv'.
1
File saved as 'Ridge_Regression_CO2_inject_rate_test_exam.csv'.
1
File saved as 'MLP_Neural_Network_CO2_inject_rate_test_exam.csv'.
1
File saved as 'Elastic_Net_CO2_inject_rate_test_exam.csv'.
1
File saved as 'Support_Vector_Regression_CO2_inject_rate_test_exam.csv'.


# WE Are done.